In [60]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
from torch import nn
import numpy as np

In [61]:
train_csv_file_path = './part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv'
train_raw_data = pd.read_csv(train_csv_file_path)

test_csv_file_path = './part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv'
test_raw_data = pd.read_csv(test_csv_file_path)

In [62]:
# iloc: integer location [row, column]
# store only data (not column name)
# use iloc, not just slicing.
train_data = train_raw_data.iloc[1:, :-1]
train_label = train_raw_data.iloc[:, -1]

test_data = test_raw_data.iloc[1:, :-1]
test_label = test_raw_data.iloc[:, -1]

# print(train_raw_data.columns) has label data

# subsetting from one dataset is done with sklearn's train_test_split
# features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Check if the two labels has the same content

In [63]:
train_unique_label = np.sort(train_label.unique())
test_unique_label = np.sort(test_label.unique())

In [64]:
print(True) if len(train_unique_label) == len(test_unique_label) and np.array_equal(train_unique_label, test_unique_label) else print(False)

True


# Check the distribution of each category

In [65]:
print("training set distribution")
train_label_counts = train_raw_data['label'].value_counts()
print(train_label_counts)

print("testing set distribution")
test_label_counts = test_raw_data['label'].value_counts()
print(test_label_counts)

training set distribution
label
DDoS-ICMP_Flood            36554
DDoS-UDP_Flood             27626
DDoS-TCP_Flood             23149
DDoS-PSHACK_Flood          21210
DDoS-SYN_Flood             20739
DDoS-RSTFINFlood           20669
DDoS-SynonymousIP_Flood    18189
DoS-UDP_Flood              16957
DoS-TCP_Flood              13630
DoS-SYN_Flood              10275
BenignTraffic               5600
Mirai-greeth_flood          5016
Mirai-udpplain              4661
Mirai-greip_flood           3758
DDoS-ICMP_Fragmentation     2377
MITM-ArpSpoofing            1614
DDoS-ACK_Fragmentation      1505
DDoS-UDP_Fragmentation      1484
DNS_Spoofing                 925
Recon-HostDiscovery          697
Recon-OSScan                 517
Recon-PortScan               430
DoS-HTTP_Flood               414
VulnerabilityScan            210
DDoS-HTTP_Flood              169
DDoS-SlowLoris               106
DictionaryBruteForce          63
SqlInjection                  31
BrowserHijacking              30
CommandInje

# Divide into data and label

In [66]:
train_features = train_raw_data.drop('label', axis = 1)
train_target = train_raw_data['label']

test_features = test_raw_data.drop('label', axis = 1)
test_target = test_raw_data['label']

# For label, do label encoding

In [67]:
train_target, train_target_mapping = pd.factorize(train_target)
test_target, test_target_mapping = pd.factorize(test_target)

In [68]:
print(train_target)
print(train_target_mapping)

[0 1 2 ... 8 9 2]
Index(['DDoS-RSTFINFlood', 'DoS-TCP_Flood', 'DDoS-ICMP_Flood', 'DoS-UDP_Flood',
       'DoS-SYN_Flood', 'Mirai-greeth_flood', 'DDoS-SynonymousIP_Flood',
       'Mirai-udpplain', 'DDoS-SYN_Flood', 'DDoS-PSHACK_Flood',
       'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'BenignTraffic', 'MITM-ArpSpoofing',
       'DDoS-ACK_Fragmentation', 'Mirai-greip_flood', 'DoS-HTTP_Flood',
       'DDoS-ICMP_Fragmentation', 'Recon-PortScan', 'DNS_Spoofing',
       'DDoS-UDP_Fragmentation', 'Recon-OSScan', 'XSS', 'DDoS-HTTP_Flood',
       'Recon-HostDiscovery', 'CommandInjection', 'VulnerabilityScan',
       'DDoS-SlowLoris', 'Backdoor_Malware', 'BrowserHijacking',
       'DictionaryBruteForce', 'SqlInjection', 'Recon-PingSweep',
       'Uploading_Attack'],
      dtype='object')


# Make custom dataset

In [69]:
train_X_tensor = torch.FloatTensor(train_features.values)
train_y_tensor = torch.from_numpy(train_target)

test_X_tensor = torch.FloatTensor(test_features.values)
test_y_tensor = torch.from_numpy(test_target)

In [70]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        # X is features, y is targets
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        a = self.X[idx]
        b = self.y[idx]
        return a, b

In [71]:
train_dataset = CustomDataset(train_X_tensor, train_y_tensor)
test_dataset = CustomDataset(test_X_tensor, test_y_tensor)

# Set hyperparameters

In [72]:
batch_size = 32

In [73]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size) #shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# set device

In [74]:
# setting up the device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")
# Using mps device

Using mps device


# define model

In [75]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(46,46),
            nn.ReLU(),
            nn.Linear(46,34)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device=device)

print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=46, out_features=46, bias=True)
    (1): ReLU()
    (2): Linear(in_features=46, out_features=34, bias=True)
  )
)


# define loss fucntion and optimizer

In [76]:
# Optimizing the model parameters
# To train a model, we need a loss function and an optimizer.
lr = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# define train and test function

In [77]:
from torch.autograd import Variable
# In a single training loop, the model makes predictinos on the training dataset (fed to it in batches)
# and backpropagates the predictino error to adjust the model's parameters
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute predictino error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 20000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}|{size:>5d}]")

# We also check the model's performance against the test dataset to ensure it is learning.
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

# train the model

In [78]:
# The training process is conducted over several iterations (epochs).
# Durin each epoch, the model learns parameters to make better predictions.
# Print the model's accuracy and loss at each epoch.
# see the accuracy increase and the loss cecrease with every epoch.
# the train and test takes approximately [6] mins in M1 mac (mps).

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------------------
loss: 3842386.250000 [   32|238687]
Test Error: 
 Accuracy: 7.7%, Avg loss: 3.376587

Epoch 2
--------------------------------
loss: 3.220924 [   32|238687]
Test Error: 
 Accuracy: 7.7%, Avg loss: 3.309892

Epoch 3
--------------------------------
loss: 3.054395 [   32|238687]
Test Error: 
 Accuracy: 7.7%, Avg loss: 3.283809

Epoch 4
--------------------------------
loss: 2.963405 [   32|238687]
Test Error: 
 Accuracy: 7.7%, Avg loss: 3.271794

Epoch 5
--------------------------------
loss: 2.908771 [   32|238687]
Test Error: 
 Accuracy: 7.7%, Avg loss: 3.264430

Epoch 6
--------------------------------
loss: 2.872543 [   32|238687]
Test Error: 
 Accuracy: 7.7%, Avg loss: 3.258856

Epoch 7
--------------------------------
loss: 2.846604 [   32|238687]
Test Error: 
 Accuracy: 7.7%, Avg loss: 3.253622

Epoch 8
--------------------------------
loss: 2.826982 [   32|238687]
Test Error: 
 Accuracy: 7.7%, Avg loss: 3.248756

Epoch 9
------------------

# save the model

In [ ]:
torch.save(model, 'model.pth')

# load model
# loaded_model = torch.load('model.pth')